In [2]:
import pandas as pd
import numpy as np
import os
import re

In [3]:
attendance = pd.DataFrame()
for year in range(18, 23):  # range is exclusive at the end
    filename = f"/Users/danielmatten/desktop/m/attendance_pub20{year}.sas7bdat"
    try:
        # Read SAS file
        df = pd.read_sas(filename)

        # Keep only relevant columns
        df.columns = df.columns.str.lower()

        # Drop missing values

        
        # Append to the main DataFrame
        attendance = pd.concat([attendance, df], ignore_index=True)
        print("{year} done")

    except FileNotFoundError:
        print(f"File {filename} not found. Skipping.")
    except KeyError:
        print(f"Required columns not found in {filename}. Skipping.")
def clean_bytes(val):
    if isinstance(val, bytes):
        return str(val)[2:-1]  # str(b'xyz') => "b'xyz'" → "xyz"
    return val
for col in attendance.columns:
    attendance[col] = attendance[col].apply(clean_bytes)
    print(col)
    
attendance['schl_yr'] = attendance['schl_yr'].str.split('-').str[1].astype(int)
attendance.dropna(subset=['mastid'], inplace=True)
#attendance.to_csv("/Users/adamcartwright/NCERDC-MODEL/data/attendance_master.csv",index=False)

{year} done
{year} done
File /Users/danielmatten/desktop/m/attendance_pub2020.sas7bdat not found. Skipping.
{year} done
{year} done
schl_yr
dayp
daya
daye
lea
schlcode
mastid


OSError: Cannot save file into a non-existent directory: '/Users/adamcartwright/NCERDC-MODEL/data'

In [24]:
exit_ids = pd.read_csv('../data/exit_mastids.csv')
# If squeeze doesn’t work (Pandas 2.x), just:
# Then use to filter another DataFrame
filtered = attendance.merge(exit_ids,on='mastid',how='inner')
filtered = filtered[filtered['mastid'].isin(exit_ids['mastid'])]


In [25]:
# Make sure both columns are numeric (just in case)
filtered['effective_g9year'] = pd.to_numeric(filtered['effective_g9year'], errors='coerce')
filtered['schl_yr'] = pd.to_numeric(filtered['schl_yr'], errors='coerce')

# Calculate grade
filtered['grade'] = filtered['effective_g9year'] - filtered['schl_yr'] + 9

# Drop the original columns
filtered = filtered.drop(columns=['schl_yr', 'effective_g9year'])


In [26]:
filtered = filtered[(filtered['grade'] >= 6) & (filtered['grade'] <= 9)] #we need a sample that will be well filled out, and i dont want to mess with more data

In [28]:
# Ensure daya and daye are numeric
filtered['daya'] = pd.to_numeric(filtered['daya'], errors='coerce')
filtered['daye'] = pd.to_numeric(filtered['daye'], errors='coerce')

# Compute absent percent
filtered['absent_pc'] = filtered['daya'] / filtered['daye']

# Filter to grades 6 through 9 only
grade_range = [6, 7, 8, 9]
filtered = filtered[filtered['grade'].isin(grade_range)]

# Count how many distinct grades each student has in that range
grade_counts = filtered.groupby('mastid')['grade'].nunique()

# Keep students with 3 or more distinct grades in that range
valid_mastids = grade_counts[grade_counts >= 3].index

# Filter main DataFrame to only those students
filtered = filtered[filtered['mastid'].isin(valid_mastids)]

# Group by mastid and grade and take mean absent_pc
agg_df = filtered.groupby(['mastid', 'grade'])['absent_pc'].mean().reset_index()

# Now pivot safely
pivot_df = agg_df.pivot(index='mastid', columns='grade', values='absent_pc')
pivot_df.columns = [f'absent_pc_grade_{g}' for g in pivot_df.columns]
pivot_df = pivot_df.reset_index()





In [30]:
# Strip '.0' from all column names
pivot_df.columns = [
    col.replace('.0', '') if isinstance(col, str) else col
    for col in pivot_df.columns
]


In [31]:
# Impute grade 7 if missing
pivot_df['absent_pc_grade_7'] = pivot_df['absent_pc_grade_7'].fillna(
    (pivot_df['absent_pc_grade_6'] + pivot_df['absent_pc_grade_8']) / 2
)

# Impute grade 8 if missing
pivot_df['absent_pc_grade_8'] = pivot_df['absent_pc_grade_8'].fillna(
    (pivot_df['absent_pc_grade_7'] + pivot_df['absent_pc_grade_9']) / 2
)

pivot_df.dropna(inplace=True)

In [32]:
pivot_df.to_csv("../data/attendance_mastid.csv",index=False)